In [171]:
from dotenv import load_dotenv
import os

load_dotenv()

if os.getenv("GROQ_API_KEY"):
    print('Groq API Key loaded successfully')
else:
    print('Groq API Key loading failed, please make sure the .env file exists and the spelling is correct')

if os.getenv("TAVILY_API_KEY"):
    print('Tavily API Key loaded successfully')
else:
    print('Tavily API Key loading failed, please make sure the .env file exists and the spelling is correct')

Groq API Key loaded successfully
Tavily API Key loaded successfully


In [172]:
from termcolor import colored

LLM_OUTPUT = 'cyan'

TEAM_RED = 'red'
TEAM_BLUE = 'blue'
BOMB = 'grey'
NEUTRAL = 'yellow'


In [173]:
# read file wordlist-eng.txt
wordlist = open("wordlist-eng.txt", "r")
words = wordlist.readlines()
wordlist.close()

# remove newline characters
words = [word.strip() for word in words]


print(words)

['AFRICA', 'AGENT', 'AIR', 'ALIEN', 'ALPS', 'AMAZON', 'AMBULANCE', 'AMERICA', 'ANGEL', 'ANTARCTICA', 'APPLE', 'ARM', 'ATLANTIS', 'AUSTRALIA', 'AZTEC', 'BACK', 'BALL', 'BAND', 'BANK', 'BAR', 'BARK', 'BAT', 'BATTERY', 'BEACH', 'BEAR', 'BEAT', 'BED', 'BEIJING', 'BELL', 'BELT', 'BERLIN', 'BERMUDA', 'BERRY', 'BILL', 'BLOCK', 'BOARD', 'BOLT', 'BOMB', 'BOND', 'BOOM', 'BOOT', 'BOTTLE', 'BOW', 'BOX', 'BRIDGE', 'BRUSH', 'BUCK', 'BUFFALO', 'BUG', 'BUGLE', 'BUTTON', 'CALF', 'CANADA', 'CAP', 'CAPITAL', 'CAR', 'CARD', 'CARROT', 'CASINO', 'CAST', 'CAT', 'CELL', 'CENTAUR', 'CENTER', 'CHAIR', 'CHANGE', 'CHARGE', 'CHECK', 'CHEST', 'CHICK', 'CHINA', 'CHOCOLATE', 'CHURCH', 'CIRCLE', 'CLIFF', 'CLOAK', 'CLUB', 'CODE', 'COLD', 'COMIC', 'COMPOUND', 'CONCERT', 'CONDUCTOR', 'CONTRACT', 'COOK', 'COPPER', 'COTTON', 'COURT', 'COVER', 'CRANE', 'CRASH', 'CRICKET', 'CROSS', 'CROWN', 'CYCLE', 'CZECH', 'DANCE', 'DATE', 'DAY', 'DEATH', 'DECK', 'DEGREE', 'DIAMOND', 'DICE', 'DINOSAUR', 'DISEASE', 'DOCTOR', 'DOG', 'DRAFT',

In [174]:
import re
import httpx
import os

from langchain_groq import ChatGroq
load_dotenv()

chat_model = ChatGroq(
    model="llama-3.1-70b-versatile",
    temperature=0,
    max_tokens=None,
    timeout=None,
    max_retries=2,
)

In [175]:
import random

In [176]:
from langgraph.graph import StateGraph, END
from typing import TypedDict, Annotated
import operator
from langchain_core.messages import AnyMessage, SystemMessage, HumanMessage, ToolMessage
from langchain_groq import ChatGroq

In [177]:
class SpymasterState(TypedDict):
    messages: Annotated[list[AnyMessage], operator.add]
    clue: str
    n_words: int

class OperativeState(TypedDict):
    messages: Annotated[list[AnyMessage], operator.add]
    clue: str
    n_words: int
    guess_number: int
    guesses: Annotated[list[str], operator.add]

chat_model = ChatGroq(
    model="llama-3.1-70b-versatile",
    temperature=0,
    max_tokens=None,
    timeout=None,
    max_retries=2,
)

In [178]:
class Board:
    def __init__(self,word_list,starting_team,random_state=42):
        random.seed(random_state)
        self.random_state = random_state
        self.word_list = word_list
        self.starting_team = starting_team

        self._prepare_grid()

    def _prepare_grid(self):
        words = random.sample(self.word_list, 25)

        n_words_team_R = 9 if self.starting_team == "R" else 8
        n_words_team_B = 9 if self.starting_team == "B" else 8
        n_words_bomb = 1
        chosen_words = random.sample(words, n_words_team_R + n_words_team_B + n_words_bomb)

        words_team_R = chosen_words[:n_words_team_R]
        words_team_B = chosen_words[n_words_team_R:n_words_team_R + n_words_team_B]
        words_bomb = chosen_words[n_words_team_R + n_words_team_B:]

        self.grid = {
            word: {
                "team":"R" if word in words_team_R else "B" if word in words_team_B else "X" if word in words_bomb else "N",
                "revealed":False
                } 
            for word in words
            }
        self.grid_hidden = {word: {"team":"?","revealed":False} for word in words}

    def get_grid(self):
        return self.grid
    
    def get_hidden_grid(self):
        self._update_hidden_grid()
        return self.grid_hidden
        
    def reveal_word(self,word):
        self.grid[word]["revealed"] = True
        self._update_hidden_grid()
        return self.grid[word]["team"] 
    
    def give_clue(self,clue,n_words):
        print(f"Spymaster gives clue: {clue} for {n_words} words")
    
    def _update_hidden_grid(self):
        for word in self.grid:
            if self.grid[word]["revealed"]:
                self.grid_hidden[word]["team"] = self.grid[word]["team"]
                self.grid_hidden[word]["revealed"] = True

    def __repr__(self) -> str:
        result = " Current Board: ".center(80,"#")
        result += "\n"
        for i,word in enumerate(self.grid):
            if i % 5 == 0 and i != 0:
                result += "\n"
            revealed = "[r]" if self.grid[word]["revealed"] else "[ ]"
            team = self.grid[word]["team"]
            team_color = TEAM_RED if team == "R" else TEAM_BLUE if team == "B" else BOMB if team == "X" else NEUTRAL
            result += colored(f"{word}{revealed}, ",team_color)
            result += "\t"
        result += "\n"
        result += "#"*80

        return result
            
            

In [179]:
class Spymaster:
    def __init__(self, model, prompt, grid: Board, team):
        self.model = model
        # prepare the StateGraph
        graph = StateGraph(SpymasterState)
        # create the nodes of the graph and corresponding behaviour
        graph.add_node("head", self.create_clue)
        graph.set_entry_point("head")
        # create the graph and turn it into a LangChain Runnable (allows to call methods like .invoke,...)
        self.graph = graph.compile()
        self.prompt = prompt
        self.team=team
        self.grid = grid

    # create a clue (all in one -> expand later on)
    def create_clue(self, state: SpymasterState):
        messages = [SystemMessage(content=self.prompt)] + [SystemMessage(str(self.grid.get_grid()))]
        retries_bad_answer = 5
        retries_model_failed = 5
        for _ in range(retries_bad_answer):
            for _ in range(retries_model_failed):
                message = self.model.invoke(messages)
                message.content = message.content.strip()
                if message.content == "":
                    print("Empty message, retrying...")
                else:
                    break # intended behavior
            if message.content == "":
                raise Exception("Model failed to generate a message after {retries_model_failed} retries")
            clue, n_words = self.extract_clue(message.content)
            print(f'Spymaster says: {colored(message.content,LLM_OUTPUT)}')
            if clue is None or n_words is None:
                print("Invalid clue, retrying...")
                messages.append(SystemMessage(content=f"Your guess was not in the correct format. Use the format '-[<clue>,<number>]-'. Only give a clue this time, do not reason again."))
            else:
                break # intended behavior
        return {
            'messages': [message],
            'clue': clue,
            'n_words': n_words}
    
    def extract_clue(self, message):
        # extract clue and number of words from message using regex of the form "-[<clue>,<number>]-"
        pattern = r"-\[(.*),(\d+)\]-"
        match = re.search(pattern, message)
        if match:
            return match.group(1), int(match.group(2))
        else:
            return None, None

In [ ]:
class Operative:
    def __init__(self, model, system_prompt_planner, system_prompt_guesser, grid: Board, team):
        self.model = model
        self.system_prompt_planner = system_prompt_planner
        self.system_prompt_guesser = system_prompt_guesser
        self.grid = grid
        self.team = team
        


        # prepare the StateGraph
        graph = StateGraph(OperativeState)
        # create the nodes of the graph and corresponding behaviour
        graph.add_node("planner", self.planner)
        graph.add_node("prepare_guess", self.prepare_guess)
        graph.add_edge("planner", "prepare_guess")
        graph.add_conditional_edges("prepare_guess", self.new_guess_available, {True: "prepare_guess", False: END})
        
        graph.set_entry_point("planner")
        # create the graph and turn it into a LangChain Runnable (allows to call methods like .invoke,...)
        self.graph = graph.compile()

    # plan the round
    def planner(self, state: OperativeState):
        # new_info = f"""
        # The clue is: {self.clue} with {self.n_words} words. The grid is: {self.grid.get_hidden_grid()}. Assign the a percentage of confidence to each word."""
        
        messages = [SystemMessage(content=self.system_prompt_planner)] + state['messages']
        retries_model_failed = 5
        for _ in range(retries_model_failed):
            message = self.model.invoke(messages)
            message.content = message.content.strip()
            
            if message.content == "":
                print("Empty message, retrying...")
            else:
                break
        print(f"Operative Planner says: {colored(message.content,LLM_OUTPUT)}")
        return {'messages': [message],
                'guess_number': 0}
    
    # create a guess (all in one -> expand later on)
    def prepare_guess(self, state: OperativeState):
        messages = [SystemMessage(content=self.system_prompt_guesser)] + state['messages'] + [SystemMessage(content="What is your guess?")]
        retries_model_failed = 5
        retries_bad_answer = 5
        for _ in range(retries_bad_answer):
            for _ in range(retries_model_failed): 
                #print(colored(messages,'red'))
                message = self.model.invoke(messages)
                message.content = message.content.strip()
                if message.content == "":
                    print("Empty message, retrying...")
                else:
                    break
            if message.content == "":
                raise Exception(f"Model failed to generate a message after {retries_model_failed} retries")
            print(f'Operative Guesser says: {colored(message.content,LLM_OUTPUT)}')
            
            guess, errors = self.extract_guess(message.content)
            # check if guess is None
            if guess is None:
                print("Invalid guess, retrying...")
                messages.append(SystemMessage(content=f"Your last guess was invalid for the following reasons: {str(errors)}."))
            else:
                break
        
        if guess is None:
            print("Model unable to generate a valid guess, ending turn.")
            guess = "!END!"
        
        # not really correct as we dont know if the guess was correct here. However, if it was not correct, the turn will end anyway
        next_turn_message = SystemMessage(content=f"Your guess was correct! Continue guessing.")

        
        return {
            'messages': [message] + [next_turn_message],
            'guesses': [guess],
            'guess_number': state['guess_number'] + 1}
    
    def take_guess(self, state: OperativeState):
        guess = state['guesses'][-1]
        team = self.grid.reveal_word(guess)
        if team == "X":
            print(f"Operative guessed the bomb: {guess}")
        elif team == "N":
            print(f"Operative guessed a neutral word: {guess}")
        elif team == self.team:
            print(f"Operative correctly guessed a word of their team: {guess}")
        else:
            print(f"Operative guessed a word of the other team: {guess}")

        return team == self.team # return True if the guess was correct, False otherwise
    
    def new_guess_available(self, state: OperativeState):
        if state['guesses'][-1] == "!END!":
            print("Ending turn.")
            return False
        guess_correct = self.take_guess(state)
        if not guess_correct:
            print("Word incorrectly guessed, ending turn.")
            return False
        if state['guess_number'] == state['n_words']+1:
            print("Maximum number of guesses reached, ending turn.")
            return False
        print("New guess available.")
        return True

    def extract_guess(self, message):
        # extract guess from message using regex of the form "-[<guess>]-"; for ending the turn it should be "-[!END!]-"
        pattern_end = r'-\[[!END!]+\]-'
        if re.search(pattern_end, message) is not None:
            return "!END!", []
        pattern = r'-\[[A-Z]+\]-'
        match = re.search(pattern, message)
        if match:
            guess = match.group(0)[2:-2]
            guess_valid = self.guess_valid(guess)
            if guess_valid[0] and guess_valid[1] :
                errors = []
            else:
                guess = None
                errors = []
                if not guess_valid[0]:
                    errors.append("Not in grid")
                if not guess_valid[1]:
                    errors.append("Already revealed")

        else:
            guess = None
            errors = ["Wrong format"]

        return guess, errors
    
    def guess_valid(self, guess):
        in_grid = guess in self.grid.get_grid()
        not_revealed = not self.grid.get_grid()[guess]["revealed"]
        return in_grid, not_revealed

In [181]:
def get_prompt_spymaster(team):
    prompt = f""" You are the Spymaster in the game Codenames and play for team {team}.
    You are given a list of words. For each word, you will see which team it belongs to and wether it was already revealed to your counterpart.
    Your task is to find words, that belong to your team, were not revealed yet and share some similiarities. You then need to find a single word that connects these words. This is called clue. You also need to provide the number of words, you tried to connect with your single word clue. Give your answer in the format: -[<clue>,<number>]-.
    Answer short and precise.
    """.strip()
    return prompt

def get_prompt_operative_planner(team):
    prompt = f"""You are an Operative in the game Codenames and play for team {team}. Your task is to prepare guesses, to help another Operative to find the words, that belong to your team.
    You are given a list of words. For each word, you will see, if it has already been revealed. You are also given a clue and a number. Using this clue shortly reason about the similarity and give a probability score for each word. 
    """.strip()
    return prompt

def get_prompt_operative_prepareGuess(team):
    prompt = f"""You are an Operative in the game Codenames and play for team {team}.
    You are given a list of words. For each word, you will see, if it has already been revealed. You are also given a clue and a number. Another Operative has assigned probabilities to each word of the list, that has not yet been revealed, that the clue is related to it. You can guess one word at a time. If you you are correct, you can continue guessing. If you are wrong, your turn ends. 
    You can guess by providing a word in the format -[<guess>]-.
    If you are unsure, you can end your turn by guessing "!END!".
    Answer short and precise.
    """.strip()
    return prompt

In [182]:
class Match:
    def __init__(self,words, starting_team="B"):
        self.board = Board(words, starting_team)

        self.current_team = starting_team # R or B
        #self.current_role = "spymaster" # spymaster or operative

        self.spymaster_red = Spymaster(chat_model, get_prompt_spymaster(self.current_team), self.board, "R")
        self.operative_red = Operative(chat_model, get_prompt_operative_planner("R"), get_prompt_operative_prepareGuess("R"), self.board, "R")
        self.spymaster_blue = Spymaster(chat_model, get_prompt_spymaster(self.current_team), self.board, "B")
        self.operative_blue = Operative(chat_model, get_prompt_operative_planner("B"), get_prompt_operative_prepareGuess("B"), self.board, "B")

    def game_loop(self):
        while True:
            print(f" Current Team: {self.current_team} ".center(80, "*"))
            print(self.board)
            print(" Turn of Spymaster ".center(80, "-"))

            # turn of spymaster
            messages = [SystemMessage(content=f"The grid is: {self.board.get_grid()}. Create a clue for your team.")]
            if self.current_team == "R":
                result = self.spymaster_red.graph.invoke({"messages": messages})
            else:
                result = self.spymaster_blue.graph.invoke({"messages": messages})
            clue = result['clue']
            n_words = result['n_words']
            print(f"Team {self.current_team} spymaster gives clue: {clue} for {n_words}")

            print(" Turn of Operative ".center(80, "-"))

            # turn of operative
            messages = [SystemMessage(content=f"The grid is: {self.board.get_hidden_grid()}. The clue is: {clue} for {n_words} words.")]
            current_operative = self.operative_red if self.current_team == "R" else self.operative_blue
            

            result = current_operative.graph.invoke(
                {
                    "messages": messages,
                    "clue": clue,
                    "n_words": n_words
                }
            )

            # check if game is over
            game_over = self.check_game_over(self.current_team)
            if game_over:
                print(f"Team {game_over} wins!")
                return game_over # representing the winning team
            
            # switch teams
            if self.current_team == "R":
                self.current_team = "B"
            else:
                self.current_team = "R"
            
    def check_game_over(self, current_team):
        team_R_revealed = [self.board.grid[word]["revealed"] for word in self.board.grid if self.board.grid[word]["team"] == "R"]
        team_B_revealed = [self.board.grid[word]["revealed"] for word in self.board.grid if self.board.grid[word]["team"] == "B"]
        bomb_revealed = [self.board.grid[word]["revealed"] for word in self.board.grid if self.board.grid[word]["team"] == "X"]

        if all(team_R_revealed):
            return "R"
        elif all(team_B_revealed):
            return "B"
        elif any(bomb_revealed):
            return not current_team # the other team wins if bomb was revealed
        else:
            return False

In [183]:
match = Match(words)
winner = match.game_loop()

******************************* Current Team: B ********************************
################################ Current Board: ################################
SOLDIER[ ], 	CARROT[ ], 	ATLANTIS[ ], 	UNDERTAKER[ ], 	FRANCE[ ], 	
FAN[ ], 	DWARF[ ], 	CHOCOLATE[ ], 	TUBE[ ], 	CANADA[ ], 	
STREAM[ ], 	POST[ ], 	BRIDGE[ ], 	SATELLITE[ ], 	MEXICO[ ], 	
BALL[ ], 	BACK[ ], 	BUFFALO[ ], 	DRILL[ ], 	ENGLAND[ ], 	
PIANO[ ], 	SCREEN[ ], 	AUSTRALIA[ ], 	RABBIT[ ], 	DEGREE[ ], 	
################################################################################
------------------------------ Turn of Spymaster -------------------------------
Spymaster says: -[FIGHTER,3]-
Team B spymaster gives clue: FIGHTER for 3
------------------------------ Turn of Operative -------------------------------
Operative Planner says: Based on the clue 'FIGHTER' for 3 words, I'll analyze each word and provide a probability score for each word.

1. SOLDIER: High probability (0.8) - A soldier is a type of fighter, and the 

In [ ]:
test = '-[!END!]-'
pattern_end = r'-\[[!END!]+\]-'
match = re.search(pattern_end, test)
if match

<re.Match object; span=(0, 9), match='-[!END!]-'>


In [187]:
print(match)

None
